In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = "filepath",
        save_best_only = True,
        monitor = 'val_loss'
    )
]

In [ ]:
test_model = keras.model.load_model('name')
test_loss, test_acc = test_model.evaluate (test_dataset)

In [ ]:
# Data Augmentation

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2)
    ]
)

In [ ]:
plt.figure(figsize=(10,10))
for images, _ in train_dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(augmented_images[0].numpy().astype('unit8'))
        plt.axis('off')

In [ ]:
inputs = keras.Input(shape = (180, 180, 3))
x = data_augmentation(input)
x = layers.Rescaling(1./255)(x)
x = Conv2D(filters = 32, kernel_size=3, activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 128, kernel_size = 3, activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 256, kernel_size = 3, activation = 'relu')(x)
x = layers.MaxPooling2D(pool_size = 2)(x)
x = layers.Conv2D(filters = 256, kernel_size = 3, activation = 'relu')(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model = keras.Model (inputs = inputs, outputs = outputs)
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath = 'filepath',
        save_best_only = True,
        monitor = 'val_loss'
    )
]

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (180,180,3)
) # ready network
# VGG16 — это популярная архитектура сверточной нейронной сети, разработанная для классификации изображений. 
# Она была обучена на большом наборе данных ImageNet, содержащем миллионы изображений, и может классифицировать их на 1000 классов.
# Эта модель используется в задачах трансферного обучения (transfer learning), 
# что позволяет перенести знания, полученные при обучении на одном наборе данных, на решение другой задачи.

In [ ]:
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

In [ ]:
train_features, train_labels = get_features_and_labels (train_dataset)
# same for test and validation features

In [ ]:
inputs = keras.Input(shape = (5,5,512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='feature_extraction.keras',
        save_best_only = True,
        monitor = 'val_loss'
    )
]